## NLP - Lesson 5 - LLM based Sentiment Analysis

Keeping up with the current trend, LLMs is the hottest thing in the AI industry right now. This is a basic notebook which can be used to learn how to read and use any pre-saved LLM from the **hugging face** library - **Transformers**. I have used the below hugging face article to read about the usage of a few pre-saved LLMs. Since our problem is to classify disaster based tweets in POSITIVE or NEGATIVE (disaster and no-disaster), we will be using an LLM trained on tweets to classify them as sentiments. Now let's start with the notebook.

Source - [Hugging-Face-Article](https://huggingface.co/blog/sentiment-analysis-python)

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [3]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use mps:0


[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [4]:
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
specific_model(data)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Device set to use mps:0


[{'label': 'POS', 'score': 0.9916695356369019},
 {'label': 'NEG', 'score': 0.9806600213050842}]

In [5]:
data = pd.read_csv("./data/train.csv")

train, test = train_test_split(data, test_size=0.25, random_state=100)
train.shape, test.shape

((5709, 5), (1904, 5))

In [6]:
train.head(3)

,id,keyword,location,text,target
3398,4866,explode,NaN,Learn How I Gained Access To The Secrets Of Th...,0
7330,10490,wildfire,Vail Valley,We should all have a fire safety plan. RT @Mat...,0
4903,6979,massacre,Cimerak - Pangandaran,Review: Dude Bro Party Massacre III http://t.c...,0


In [7]:
test.head(3)

,id,keyword,location,text,target
3999,5680,floods,Estados Unidos,Typhoon Soudelor approaches after 7 killed 2 m...,1
1849,2659,crush,EastAtlanta ??#WestGeorgia'18,WCE I can't even lie even tho I can't stand he...,0
5254,7514,oil%20spill,NaN,SYD traffic HAZARD Oil spill - BANKSTOWN Stace...,1


In [8]:
%%time

train['sentiment_distilbert'] = train['text'].map(lambda x : sentiment_pipeline(x)[0]['label'])
test['sentiment_distilbert'] = test['text'].map(lambda x : sentiment_pipeline(x)[0]['label'])
train['sentiment_bertweet']= train['text'].map(lambda x : specific_model(x)[0]['label'])
test['sentiment_bertweet']= test['text'].map(lambda x : specific_model(x)[0]['label'])

CPU times: user 2min 10s, sys: 27.1 s, total: 2min 37s
Wall time: 2min 40s


In [9]:
print(train['sentiment_distilbert'].value_counts(), test['sentiment_distilbert'].value_counts())

sentiment_distilbert
NEGATIVE    4810
POSITIVE     899
Name: count, dtype: int64 sentiment_distilbert
NEGATIVE    1586
POSITIVE     318
Name: count, dtype: int64


In [10]:
print(train['sentiment_bertweet'].value_counts(), test['sentiment_bertweet'].value_counts())

sentiment_bertweet
NEU    2606
NEG    2357
POS     746
Name: count, dtype: int64 sentiment_bertweet
NEU    877
NEG    762
POS    265
Name: count, dtype: int64


In [11]:
train['pred_distilbert'] = np.where(train['sentiment_distilbert']=='NEGATIVE',1,0)
test['pred_distilbert'] = np.where(test['sentiment_distilbert']=='NEGATIVE',1,0)
train['pred_bertweet'] = np.where(train['sentiment_bertweet']=='POS',0,1)
test['pred_bertweet'] = np.where(test['sentiment_bertweet']=='POS',0,1)

In [12]:
def get_metrics(y, pred, dataset='given', ret_metrics=False):
    print("#"*125)
    prec = precision_score(y, pred)
    rec = recall_score(y, pred)
    f1 = f1_score(y, pred)
    # print(f"AUC for {dataset} : ",auc)
    print(f"Precision for {dataset} : ",prec)
    print(f"Recall for {dataset} : ",rec)
    print(f"F1 Score for {dataset} : ",f1)
    print("#"*125)
    if ret_metrics:
        return auc, prec, rec, f1

get_metrics(train['target'],train['pred_distilbert'],dataset='Train Distilbert')
get_metrics(test['target'],test['pred_distilbert'],dataset='Test Distilbert')
get_metrics(train['target'],train['pred_bertweet'],dataset='Train Bertweet')
get_metrics(test['target'],test['pred_bertweet'],dataset='Test Bertweet')

#############################################################################################################################
Precision for Train Distilbert :  0.4602910602910603
Recall for Train Distilbert :  0.8941841680129241
F1 Score for Train Distilbert :  0.6077408729069448
#############################################################################################################################
#############################################################################################################################
Precision for Test Distilbert :  0.44703656998738966
Recall for Test Distilbert :  0.8918238993710692
F1 Score for Test Distilbert :  0.5955480890382192
#############################################################################################################################
#############################################################################################################################
Precision for Train Bertweet :  0.47209349183961313
Recall for

### Conclusion

We see that a pre-trained LLM could not get a good precision but still did very well with respect to recall. If we adjust thresholds for probabilities to optimise and balance between precision and recall, maybe we will be able to reach around 80%. In the next notebook, we will be building a PEFT (Parameter Efficient Fine Tuning) model which will be trained on this particular dataset.